In [96]:
!pip3 install mesa

In [97]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que es posible tener multiples agentes una sola celda elegimos `MultipleGrid`.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import math
import random

In [98]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        hasRobotAgent = False
        for content in cell_content:
            if isinstance(content, robotAgent):
                if content.carrying == True:
                  grid[x][y] = 7
                else:
                  grid[x][y] = 6
                hasRobotAgent = True
            elif isinstance(content, boxAgent):
                if hasRobotAgent != True:
                  grid[x][y] = grid[x][y] + 1
            else:
                grid[x][y] = 0;
    return grid

def arrayDiff(a, b):
    return [x for x in a if x not in b]

class robotAgent(Agent):
    def __init__(self, unique_id, pos, model):
      super().__init__(unique_id, model)
      self.currentBox = boxAgent
      self.pos = pos
      self.carrying = False
      self.wait = False
      self.lastLoc = (1,0)
      self.lastDirection = 1

    def getNewLoc(self, loc):
      newPos = (0,0)
      if self.pos[0] > loc[0]:
        if self.pos[1] > loc[1]:
          newPos = (self.pos[0]-1, self.pos[1]-1)
        elif self.pos[1] < loc[1]:
          newPos = (self.pos[0]-1, self.pos[1]+1)
        else:
          newPos = (self.pos[0]-1, self.pos[1])
      elif self.pos[0] < loc[0]:
        if self.pos[1] > loc[1]:
          newPos = (self.pos[0]+1, self.pos[1]-1)
        elif self.pos[1] < loc[1]:
          newPos = (self.pos[0]+1, self.pos[1]+1)
        else:
          newPos = (self.pos[0]+1, self.pos[1])
      else:
        if self.pos[1] > loc[1]:
          newPos = (self.pos[0], self.pos[1]-1)
        elif self.pos[1] < loc[1]:
          newPos = (self.pos[0], self.pos[1]+1)
        else:
          newPos = (self.pos[0], self.pos[1])
      return newPos

    def moveRobot(self):
      newPos = (0,0)
      if not self.pos == self.lastLoc:
        newPos = self.getNewLoc(self.lastLoc)
      else:
        if self.pos[1] == self.model.width - 1 and self.lastDirection == 1:
            newPos = (self.pos[0]+1, self.model.width - 1)
            self.lastDirection = 0
        elif self.pos[1] == 0 and self.lastDirection == 0:
            newPos = (self.pos[0]+1, 0)
            self.lastDirection = 1
        elif self.pos == (self.model.height - 1, self.model.width - 1):
          self.lastLoc = (0,0)
          newPos = (self.pos[0]-1, self.pos[1]-1)
        else:
          if self.lastDirection == 1:
            newPos = (self.pos[0], self.pos[1]+1)
          else:
            newPos = (self.pos[0], self.pos[1]-1)
        self.lastLoc = newPos
      tempCell = self.model.grid.get_cell_list_contents([newPos])
      numRobotsOnCell = len([obj for obj in tempCell if isinstance(obj, robotAgent)])
      if not numRobotsOnCell == 0:
        hasRobot = True
        tempCount = 0
        nextMoves = self.model.grid.get_neighborhood(self.pos, True, False)
        random.shuffle(nextMoves)
        while hasRobot and tempCount < len(nextMoves):
          tempCell1 = self.model.grid.get_cell_list_contents([nextMoves[tempCount]])
          numRobotsOnCell1 = len([obj for obj in tempCell1 if isinstance(obj, robotAgent)])
          if numRobotsOnCell1 == 0:
            newPos = nextMoves[tempCount]
            hasRobot = False
          tempCount += 1


      self.model.grid.move_agent(self, newPos)          
      

    def step(self):
      if self.wait == False:
        cellCoords = self.model.grid.get_neighborhood(self.pos, False, True)
        random.shuffle(cellCoords)
        
        i = 0
        while i < len(cellCoords) and self.carrying == False:
          cell = self.model.grid.get_cell_list_contents(cellCoords[i])
          for content in cell:
            if isinstance(content, boxAgent):
              if content.carrierId == 0 and content.movable == True:
                self.currentBox = content
                self.carrying = True;
                self.currentBox.carrierId = self.unique_id
                self.currentBox.moveAgent(self.pos)
          i += 1

        if self.carrying:
          if self.currentBox.carrierId == self.unique_id and self.currentBox.movable == True:
            self.currentBox.moveAgent(self.pos)
            if self.pos == self.model.centerCoord:
              self.currentBox.moveAgent((self.pos[0]-1, self.pos[1]))
              self.currentBox.movable = False
              self.currentBox = None
              self.carrying = False
              self.model.numBoxesPlaced += 1
            else:
              newPos = self.getNewLoc(self.model.centerCoord)
              tempCell = self.model.grid.get_cell_list_contents([newPos])
              numRobotsOnCell = len([obj for obj in tempCell if isinstance(obj, robotAgent)])
              if numRobotsOnCell == 0:
                self.model.grid.move_agent(self, newPos)              
          else:
            self.currentBox = None
            self.carrying = False
            self.moveRobot()
        else:
          self.moveRobot()

        

class boxAgent(Agent):    
    def __init__(self, unique_id, model, pos):
        super().__init__(unique_id, model)
        self.pos = pos
        self.carrierId = 0
        self.movable = True

    def moveAgent(self, newPos):
        self.model.grid.move_agent(self, newPos)
class warehouseModel(Model):

    # Define el modelo de la actividad.

    def __init__(self, width, height, numBoxes):
        super().__init__()
        self.height = height
        self.width = width
        self.centerCoord = (1, 0)
        self.grid = MultiGrid(self.width, self.height, False)
        self.schedule = SimultaneousActivation(self)

        self.numBoxes = numBoxes
        self.numBoxesPlaced = 0
        self.numStepsDone = 0;
        
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
                
        tilesAll = random.sample(range(0, ((self.height * self.width)-1)), (self.numBoxes + 5))
        tilesRobots = random.sample(tilesAll, 5)
        tilesBoxes = arrayDiff(tilesAll, tilesRobots)

        # Crear tiles sucias y limpias
        countTiles = 0
        for agent, x, y in self.grid.coord_iter():
          if countTiles in tilesBoxes:
            boxTile = boxAgent(self.next_id(), self, (y,x))
            self.grid.place_agent(boxTile, (y,x))
            self.schedule.add(boxTile)
          elif countTiles in tilesRobots:
            tempRobot = robotAgent(self.next_id(), (y,x), self)
            self.grid.place_agent(tempRobot, (y,x))
            self.schedule.add(tempRobot)
          countTiles += 1

            
        
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        cellCenter = self.grid.get_cell_list_contents([self.centerCoord])
        RobotsOnCell = [obj for obj in cellCenter if isinstance(obj, robotAgent)]
        numRobotsOnCell = len(RobotsOnCell)
        if numRobotsOnCell > 1:
          for content in RobotsOnCell:
            content.wait = True
          RobotsOnCell[0].wait = False


        cellPlace = self.grid.get_cell_list_contents([(self.centerCoord[0]-1, self.centerCoord[1])])
        numBoxesOnCell = len([obj for obj in cellPlace if isinstance(obj, boxAgent) and obj.movable == False])
        if numBoxesOnCell > 4:
          if self.centerCoord[1] == self.width - 1:
            self.centerCoord = (self.centerCoord[0]+1, 0)
          else:
            self.centerCoord = (self.centerCoord[0], self.centerCoord[1]+1)


        
        self.datacollector.collect(self)
        self.schedule.step()
        self.numStepsDone += 1


        if numRobotsOnCell > 1:
          for content in RobotsOnCell:
            content.wait = False
        

        return [self.numStepsDone, self.numBoxesPlaced]

In [99]:
# Definimos el tamaño del Grid
m = 10
n = 10

percentageBoxTiles = 43 # Porciento
# Tiempo de execucion maximo
maxTime = 1 # En segundos
# Numero de Aspiradoras
numCleaners = 10

# Registramos el tiempo de inicio y limite
startTime = time.time()
limitTime = datetime.timedelta(seconds=maxTime)

# Variables temporales
temp = [1,0]
numBoxTiles = int((m * n) * (percentageBoxTiles/100))
extraSteps = 1

#Inicializamos el modelo y realizamos los steps
model = warehouseModel(m, n, numBoxTiles)
while (datetime.timedelta(seconds=(time.time() - startTime)) < limitTime and not extraSteps == 0):
    if temp[1] == numBoxTiles:
        extraSteps -= 1
    temp = model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - startTime))))
print('Steps:', temp[0])
print('Boxes Placed:', temp[1])


Tiempo de ejecución: 0:00:00.043039
Steps: 193
Boxes Placed: 43


In [100]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [101]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = axs.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [102]:
anim